# Structure Optimization

In [ ]:
%matplotlib notebook
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
from potentials import *
from distances import *
from sampling import *

In [ ]:
import matplotlib
import matplotlib.cm as cm

delta = 0.025
x = np.arange(-1.0, 1.0, delta)
y = np.arange(-1.0, 1.0, delta)
X, Y = np.meshgrid(x, y)
XY = np.append(X[:, :, None], Y[:, :, None], axis=-1)
Z = potentials.harmonic(XY)

fig, ax = plt.subplots()
ax.contour(X, Y, Z)
#ax.clabel(CS, inline=1, fontsize=10)
#ax.set_title('Simplest default with labels')

In [ ]:
#xinit = np.array([[1., 1], [-1, 2]])
#x = xinit
#x1 = x * 0.5
#print(x1)
#x = np.append(x[None, :, :], x1[None, :, :], axis=0)
#print(x)
#print(x[-1])
#x1 = x[-1] * 0.5
#print(x1)
#x = np.append(x,  x1[None, :, :], axis=0)
#print(x)

In [ ]:
def descent( x, a=1e-4, prec=1e-10, maxst=1e6 ):
    """Gradient Descent
    
    Arguments:
        x    (float): position vectors (dim = n x 3)
        a    (float): 'learning rate' alpha = 1e-4
        prec (float): difference between steps, precision = 1e-10
        maxst  (int): max # of steps, maxst = 1e6
    
    Output:
        x: position array,
        step: # of steps needed to converge"""
    
    x = x[None, :, :]
    step = 0
    f = gradients.harmonic( x[-1] )
    x1 = x[-1] - a * f
    
    while(step < maxst and (abs(x[-1] - x1) > prec).all()):
        x = np.append(x, x1[None, :, :], axis=0)
        f = gradients.harmonic( x[-1] )
        x1 = x[-1] - a * f
        step += 1
        
    return x, step

In [ ]:
x_init = np.array([[.8, .9], [.2, .3]])
positions, nsteps = descent(x_init, 1e-3)
print(nsteps, positions.shape)
print(positions[-1])

In [ ]:
from matplotlib import animation


import matplotlib.cm as cm
delta = 0.025
x = np.arange(-3.0, 3.0, delta)
y = np.arange(-3.0, 3.0, delta)
X, Y = np.meshgrid(x, y)
XY = np.append(X[:, :, None], Y[:, :, None], axis=-1)
Z = potentials.harmonic(XY)
fig, ax = plt.subplots(figsize=(8, 8))
ax.contour(X, Y, Z)


#fig, ax = plt.subplots(figsize=(8, 8))
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], c=colors)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
def animate(i):
    index = i
    data = positions[index]
    scat.set_offsets(data)
    ax.contour(X, Y, Z)
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=80)#, frames=750, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

In [ ]:
"""
Matplotlib Animation Example

author: Jake Vanderplas
"""

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(-1, 1), ylim=(-1, 1))
line, = ax.plot([], [], lw=2)
ax.contour(X, Y, Z)

# initialization function: plot the background of each frame
def init():
    particles.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    x = positions[i, :, 0]
    y = positions[i, :, 1]
    particles.set_data(x, y)
    return line,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
#anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

plt.show()

In [ ]:
print(np.linspace(-2, 2, 1000))

In [ ]:
print(positions[0])
print(positions[0, :, 1])